# [LAB09] 지도학습 > 분류 > 08-다중분류(2)

## #01. 준비작업

### [1] 패키지 가져오기

In [ ]:
# 라이브러리 기본 참조
from hossam import *
from pandas import DataFrame, concat
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler

# 분류모형 (대표모형 하나만 지정)
from sklearn.ensemble import RandomForestClassifier

# 불균형 데이터 처리
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline

# 로지스틱 성능평가 함수
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    roc_auc_score
)

### [2] 필요한 함수

- hs_describe()
- hs_category_describe()

### [3] 데이터 가져오기

In [ ]:
origin = load_data('wine_dataset')
origin.head()

### [4] 명목형에 대한 타입 변환

데이터 품질 확인을 위해서는 타입 변환이 수행되어야 한다.

In [ ]:
df1 = origin.copy()
df1['class'] = df1['class'].astype('category')
df1.info()

## #02. 데이터 품질 확인

### [1] 연속형 변수 품질 확인

In [ ]:
desc = hs_describe(df1)
desc

> 결측치는 없으나, 약간의 이상치가 발견된다. 로그 변환을 통해 이상치를 완화할 수 있다.

### [2] 로그변환

In [ ]:
fields = desc[desc['log_need'] != '낮음'].index.tolist()
fields

In [ ]:
df2 = df1.copy()

for col in fields:
    df2[col] = np.log1p(df2[col])

df2.head()

### [3] 명목형 변수(종속변수) 확인

In [ ]:
a, b = hs_category_describe(df2)
display(a)
display(b)

> 클래스간 비율에 큰 차이를 보이지 않는다. 데이터 증강이 필요 없을것으로 보인다. 하지만 데이터 증강후 결과를 비교해 볼 필요는 있다.

## #03. 분류모형 적합

### [1] 데이터 분할

In [ ]:
yname = "class"
x = df2.drop(columns=[yname])
y = df2[yname]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,
                                                      random_state=52)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### [2] 학습모델 구성

실습 시간을 고려하여 RandomForest 모델에 대해서만 적용한다.

In [ ]:
%%time

pipe = Pipeline([
    (
        "model",
        RandomForestClassifier(
            random_state=52,
            n_jobs=-1
        )
    ),
])

param_grid = {
    "model__n_estimators": [300, 500],
    "model__max_depth": [None, 10],
    "model__min_samples_leaf": [5, 10],
    "model__max_features": ["sqrt", 1.0],
    "model__criterion": ["gini", "entropy"],
    "model__class_weight": [None, "balanced"],
}

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

gs.fit(x_train, y_train)

estimator = gs.best_estimator_
estimator

## #04. 성능평가

### [1] 다중분류 성능평가 함수 정의

In [ ]:
def hs_cls_multi_scores(estimator, x_test, y_test):
    #-----------------------------
    # 입력값 정리
    #-----------------------------
    y_pred = estimator.predict(x_test)
    y_proba = estimator.predict_proba(x_test)
    classes = np.unique(y_test)

    #-----------------------------
    # 이항분류와 동일하게 사용가능한 지표
    #-----------------------------
    score_df = DataFrame({
        "accuracy": [accuracy_score(y_test, y_pred)],
        "precision": [precision_score(y_test, y_pred, average="macro", zero_division=0)],
        "recall": [recall_score(y_test, y_pred, average="macro", zero_division=0)],
        "f1": [f1_score(y_test, y_pred, average="macro", zero_division=0)],
    })

    #-----------------------------
    # 다중분류 전용 지표 (OvR)
    #-----------------------------
    fpr_list = []
    tnr_list = []

    for i, cls in enumerate(classes):
        # 해당 클래스 vs 나머지
        y_true_binary = (y_test == cls).astype(int)
        y_pred_binary = (y_pred == cls).astype(int)

        cm = confusion_matrix(y_true_binary, y_pred_binary, labels=[0, 1])
        TN, FP, FN, TP = cm.ravel()

        fpr = FP / (FP + TN) if (FP + TN) > 0 else np.nan
        tnr = TN / (FP + TN) if (FP + TN) > 0 else np.nan

        fpr_list.append(fpr)
        tnr_list.append(tnr)

    score_df["FPR (macro)"] = np.nanmean(fpr_list)
    score_df["TNR (macro)"] = np.nanmean(tnr_list)

    # -----------------------------
    # 다중 AUC (OvR 방식)
    # -----------------------------
    score_df["AUC (macro, OvR)"] = roc_auc_score(
        y_test,
        y_proba,
        multi_class="ovr",
        average="macro"
    )

    # -----------------------------
    # 시각화
    # -----------------------------
    # y_test를 1차원 배열로 변환
    y_test_array = np.asarray(y_test).ravel()

    for i, cls in enumerate(classes):
        # 해당 클래스 vs 나머지
        y_true_binary = (y_test_array == cls).astype(int)
        y_score = y_proba[:, i]

        # ROC 계산
        fpr, tpr, thresholds = roc_curve(y_true_binary, y_score)
        auc_score = roc_auc_score(y_true_binary, y_score)

        # -----------------------------
        # 그래프 (클래스별 하나씩)
        # -----------------------------
        fig, ax = plt.subplots(1, 1, dpi=100, figsize=(480/100, 480/100))
        sb.lineplot(x=fpr, y=tpr)
        sb.lineplot(x=[0, 1], y=[0, 1], linestyle="--")
        ax.set_xlabel("False Positive Rate", fontsize=9)
        ax.set_ylabel("True Positive Rate", fontsize=9)
        ax.set_title(f"ROC Curve - Class {cls} (AUC={auc_score:.4f})", fontsize=10)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.grid(True, alpha=0.3)
        plt.show()

    # -----------------------------
    # 결과리턴
    # -----------------------------
    return score_df

### [2] 성능평가

In [ ]:
score_df = hs_cls_multi_scores(estimator, x_test, y_test)
score_df

## #05. 데이터 증강 후 결과 비교

### [1] 데이터 증강 모델 적합

In [ ]:
sm_pipe = imbPipeline([
    ('scaler', StandardScaler()),
    ('sm', SMOTE(random_state=52, k_neighbors=5)),
    ('model', RandomForestClassifier(random_state=52, n_jobs=-1))
])

sm_param_grid = {
    "model__n_estimators": [300, 500],
    "model__max_depth": [None, 10],
    "model__min_samples_leaf": [5, 10],
    "model__max_features": ["sqrt", 1.0],
    "model__criterion": ["gini", "entropy"],
    "model__class_weight": [None, "balanced"],
}

sm_gs = GridSearchCV(
    estimator=sm_pipe,
    param_grid=sm_param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

sm_gs.fit(x_train, y_train)

rf_cls_estimator = sm_gs.best_estimator_

sm_score_df = hs_cls_multi_scores(
    rf_cls_estimator,
    x_test,
    y_test
)

rdf = concat([score_df, sm_score_df], axis=0)
rdf.index = ['기본', 'SMOTE']
rdf

> 처음 예측했던바와 같이 데이터 불균형이 심하지 않은 데이터이므로 데이터 증강의 효과는 없다.

### 참고 - 로그변환을 수행하지 않은 경우의 결과

| | accuracy | precision | recall | f1 | FPR (macro) | TNR (macro) | AUC (macro, OvR) |
|---|---|---|---|---|---|---|---|
| 기본 | 0.978 | 0.972 | 0.982 | 0.976 | 0.010 | 0.990 | 0.997 |
| SMOTE | 0.978 | 0.972 | 0.982 | 0.976 | 0.010 | 0.990 | 0.997 |